# Dakota vs Salesforce Data Clean
- USA name standardization done for both in load block
- Map state ownership for both

## Dakota Clean Up
Raw | Wirehouses | Blank Emails, Dupe Emails, Dupe Names | Clean | AUM Aggregated

# Salesforce Clean Up
Raw | Dupe Emails | Dupe Names | Empty Emails | Clean 


# Packages

In [1]:
%pip install pandas openpyxl matplotlib 
import pandas as pd
import matplotlib.pyplot as plt
import os
import difflib
from difflib import get_close_matches


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


# Load all files

In [2]:
dakota_df=pd.read_excel("../data/all_contacts/Dakota_All_Contacts.xlsx")
matched_contacts=pd.read_excel("../data/prospect/matched_contacts.xlsx")
matched_accounts=pd.read_excel("../data/prospect/matched_accounts.xlsx")
unmatched_contacts=pd.read_excel("../data/prospect/unmatched_contacts.xlsx")
unmatched_accounts=pd.read_excel("../data/prospect/unmatched_accounts.xlsx")

salesforce_df = pd.read_excel("../data/all_contacts/Salesforce_dataset.xlsx") # Josh approved
salesforce_618 = pd.read_excel("../data/all_contacts/Contact_SF_Extract_6_16_25.xlsx") # find out difference between this and the above


# Mutual Changes
- U.S Standardization
- Ownership Map

In [3]:
# Define standardization function
def standardize_country(country, state):
    if pd.isnull(country):
        country = ''
    country_upper = str(country).strip().upper()
    if country_upper in ['UNITED STATES', 'U.S.', 'USA', 'US']:
        return 'United States'
    if pd.notnull(state) and str(state).strip() != '':
        return 'United States'
    return country

# List of possible column names for country and state
country_column_candidates = [
    'Mailing Country', 'MailingCountry', 'Billing Country', 'BillingCountry',
    'Provided BillingCountry', 'Dakota Billing Country', 'Provided MailingCountry', 'Dakota Mailing Country'
]
state_column_candidates = [
    'Mailing State/Province', 'Mailing State', 'Billing State/Province', 'Billing State'
]

# Function to find the first matching column from a list of candidates
def find_column(df, candidates):
    for col in candidates:
        if col in df.columns:
            return col
    return None

# Apply standardization to a list of DataFrames
def apply_country_standardization(dfs):
    for df in dfs:
        country_col = find_column(df, country_column_candidates)
        state_col = find_column(df, state_column_candidates)
        if country_col:
            df['Mailing Country'] = df.apply(
                lambda row: standardize_country(row.get(country_col), row.get(state_col)),
                axis=1
            )

# Example usage:
apply_country_standardization([dakota_df, matched_contacts, matched_accounts, unmatched_contacts, unmatched_accounts, salesforce_df, salesforce_618])


In [4]:
# Define state → team member
state_team_map = {
    'AL': 'AG', 'AK': 'Rockefeller Asset Management', 'AZ': 'KH', 'AR': 'Rockefeller Asset Management',
    'CA': 'AP', 'CO': 'KH', 'CT': 'Rockefeller Asset Management', 'DE': 'KH', 'FL': 'Rockefeller Asset Management',
    'GA': 'AP', 'HI': 'Rockefeller Asset Management', 'ID': 'Rockefeller Asset Management', 'IL': 'MJ',
    'IN': 'KH', 'IA': 'Rockefeller Asset Management', 'KS': 'Rockefeller Asset Management', 'KY': 'AG',
    'LA': 'Rockefeller Asset Management', 'ME': 'Rockefeller Asset Management', 'MD': 'KH', 'MA': 'AP',
    'MI': 'KH', 'MN': 'Rockefeller Asset Management', 'MS': 'Rockefeller Asset Management', 'MO': 'AG',
    'MT': 'Rockefeller Asset Management', 'NE': 'Rockefeller Asset Management', 'NV': 'Rockefeller Asset Management',
    'NH': 'Rockefeller Asset Management', 'NJ': 'KH', 'NM': 'KH', 'NY': 'AG', 'NC': 'AG', 'ND': 'Rockefeller Asset Management',
    'OH': 'KH', 'OK': 'Rockefeller Asset Management', 'OR': 'Rockefeller Asset Management', 'PA': 'KH',
    'RI': 'Rockefeller Asset Management', 'SC': 'AG', 'SD': 'Rockefeller Asset Management', 'TN': 'AG',
    'TX': 'MJ', 'UT': 'KH', 'VT': 'Rockefeller Asset Management', 'VA': 'KH', 'WA': 'Rockefeller Asset Management',
    'WV': 'Rockefeller Asset Management', 'WI': 'MJ', 'WY': 'Rockefeller Asset Management', 'DC': 'KH'
}

# Define team member → OwnerID
team_ownerID_map = {
    'AG': '005a600000177CHAAY',
    'Rockefeller Asset Management': '005a6000005CXtNAAW',
    'KH': '005a6000000c1j8AAA',
    'AP': '005a6000000c1jDAAQ',
    'MJ': '005a60000043nujAAA'
}

# Example: apply to a list of dataframes
dataframes = {
    "dakota_df": dakota_df,
    "matched_contacts": matched_contacts,
    "matched_accounts": matched_accounts,
    "unmatched_contacts": unmatched_contacts,
    "unmatched_accounts": unmatched_accounts,
    "salesforce_df": salesforce_df,
    "salesforce_618": salesforce_618
}

for name, df in dataframes.items():
    if 'Company Owner' not in df.columns:
        df['Company Owner'] = None
    if 'Mailing State/Province' in df.columns:
        df['Company Owner'] = df.apply(
            lambda row: state_team_map.get(row['Mailing State/Province'], row['Company Owner'])
            if pd.isna(row['Company Owner']) else row['Company Owner'],
            axis=1
        )
    df['OwnerID'] = df['Company Owner'].map(team_ownerID_map)


# Dakota Phase 1: Wirehouse Removal

In [8]:
# Define the wirehouse map
wirehouse_map = {
    'Bank of America Merrill Lynch': 'Merrill Lynch',
    'Wells Fargo Advisors LLC': 'Wells Fargo',
    'Morgan Stanley &amp; Co.': 'Morgan Stanley',
    'J.P. Morgan Securities LLC': 'J.P. Morgan',
    'UBS Financial Services Inc.': 'UBS',
    'Ameriprise Financial Services Inc.': 'Ameriprise',
    'Raymond James Financial Services Inc.': 'Raymond James',
    'Goldman Sachs &amp; Co. LLC': 'Goldman Sachs',
    'UBS': 'UBS',
    'Morgan Stanley': 'Morgan Stanley',
    'Wells Fargo Advisors': 'Wells Fargo',
    'Wells Fargo': 'Wells Fargo',
    'Ameriprise Financial Services': 'Ameriprise',
    'Goldman Sachs International': 'Goldman Sachs',
    'Raymond James Financial Services Advisors, Inc': 'Raymond James',
    'Raymond James &amp; Associates Inc.': 'Raymond James',
    'J.P. Morgan &amp; Co.': 'J.P. Morgan',
    'BlackRock': 'BlackRock',
    'Edward D. Jones': 'Edward Jones',
    'BlackRock Alternative Investors': 'BlackRock',
    'BlackRock Alternative Investors (BAI)': 'BlackRock'
}

# Create a normalized version of the wirehouse map for matching
normalized_map = {k.lower(): v for k, v in wirehouse_map.items()}

# Fuzzy match function using normalized names
def match_wirehouse(name):
    normalized_name = name.strip().lower()
    matches = difflib.get_close_matches(normalized_name, normalized_map.keys(), n=1, cutoff=0.6)
    return normalized_map[matches[0]] if matches else None

# Apply fuzzy matching without altering the original 'Account Name'
dakota_df['wirehouse_group'] = dakota_df['Account Name'].apply(match_wirehouse)

# Count and display results
print(f"Total number of account entries: {dakota_df['Account Name'].count()}")
print("\nWirehouse group counts:")
print(dakota_df['wirehouse_group'].value_counts(dropna=False))


Total number of account entries: 167025

Wirehouse group counts:
wirehouse_group
None              88061
Wells Fargo       19609
Merrill Lynch     13203
Morgan Stanley    11246
J.P. Morgan       11034
Edward Jones       9699
UBS                6793
Ameriprise         4064
Raymond James      2805
Goldman Sachs       274
BlackRock           237
Name: count, dtype: int64


In [9]:
# Total accounts at start
total_accounts_start = len(dakota_df)

# Split into wirehouse and non-wirehouse groups
dakota_wirehouses = dakota_df[dakota_df['wirehouse_group'].notna()].copy()
dakota_non_wirehouse_df = dakota_df[dakota_df['wirehouse_group'].isna()].copy()

# Count wirehouses removed and accounts left
wirehouses_removed = len(dakota_wirehouses)
total_accounts_left = len(dakota_non_wirehouse_df)

# Output directory
output_dir = "../data/dakota_salesforce_sheets"
os.makedirs(output_dir, exist_ok=True)

# Save the wirehouse and non-wirehouse datasets with new names
dakota_wirehouses.to_excel(os.path.join(output_dir, "Wirehouse_Accounts.xlsx"), index=False)
dakota_non_wirehouse_df.to_excel(os.path.join(output_dir, "Non_Wirehouse_Accounts.xlsx"), index=False)

# Summary stats
num_unique_emails = dakota_non_wirehouse_df['Email'].nunique()
num_unique_accounts = dakota_non_wirehouse_df['Account Name'].nunique()
num_duplicate_emails = dakota_non_wirehouse_df['Email'].duplicated().sum()

print("Summary Statistics:")
print(f"Number of unique emails left: {num_unique_emails}")
print(f"Number of unique account names left: {num_unique_accounts}")
print(f"Number of duplicate emails: {num_duplicate_emails}")

print("\nAccount Totals:")
print(f"Total accounts at start: {total_accounts_start}")
print(f"Wirehouses removed: {wirehouses_removed}")
print(f"Total accounts left: {total_accounts_left}")


Summary Statistics:
Number of unique emails left: 76673
Number of unique account names left: 27872
Number of duplicate emails: 11387

Account Totals:
Total accounts at start: 167025
Wirehouses removed: 78964
Total accounts left: 88061


# Phase 2: Dupes and Blanks

In [11]:
# Total entries
total_entries = len(dakota_non_wirehouse_df)

# Create Full Name column
dakota_non_wirehouse_df['Full Name'] = (
    dakota_non_wirehouse_df['First Name'].astype(str).str.strip() + ' ' +
    dakota_non_wirehouse_df['Last Name'].astype(str).str.strip()
)

# Step 1: Identify blank emails
dakota_blanks = dakota_non_wirehouse_df[
    dakota_non_wirehouse_df['Email'].isna() |
    (dakota_non_wirehouse_df['Email'].astype(str).str.strip() == '')
]

# Remove blanks from main dataset
remaining_df = dakota_non_wirehouse_df.drop(dakota_blanks.index)

# Step 2: Identify duplicate emails (excluding blanks)
non_blank_df = remaining_df[
    ~remaining_df['Email'].isna() &
    (remaining_df['Email'].astype(str).str.strip() != '')
]
dakota_dupes = non_blank_df[non_blank_df.duplicated(subset='Email', keep=False)]
dakota_dupes = dakota_dupes.sort_values(by='Email')

# Remove duplicate emails from main dataset
remaining_df = remaining_df.drop(dakota_dupes.index)

# Step 3: Identify duplicate names (excluding blanks and duplicate emails)
dakota_name_dupes = remaining_df[
    remaining_df.duplicated(subset='Full Name', keep=False)
].sort_values(by='Full Name')

# Final cleaned dataset (not in any of the above categories)
cleaned_df = remaining_df.drop(dakota_name_dupes.index)

# Counts
num_blank = len(dakota_blanks)
num_duplicates = len(dakota_dupes)
num_name_duplicates = len(dakota_name_dupes)
num_cleaned = len(cleaned_df)

# Print summary
print("Summary:")
print(f"Total entries: {total_entries}")
print(f"Blank emails: {num_blank}")
print(f"Duplicate emails (non-blank): {num_duplicates}")
print(f"Duplicate names: {num_name_duplicates}")
print(f"Cleaned entries: {num_cleaned}")

# Save outputs
output_dir = "../data/dakota_salesforce_sheets"
os.makedirs(output_dir, exist_ok=True)

dakota_blanks.to_excel(os.path.join(output_dir, "Dakota_Blanks.xlsx"), index=False)
dakota_dupes.to_excel(os.path.join(output_dir, "Dakota_Duplicates.xlsx"), index=False)
dakota_name_dupes.to_excel(os.path.join(output_dir, "Dakota_DuplicateNames.xlsx"), index=False)
cleaned_df.to_excel(os.path.join(output_dir, "Dakota_Cleaned.xlsx"), index=False)


Summary:
Total entries: 88061
Blank emails: 10844
Duplicate emails (non-blank): 1052
Duplicate names: 5092
Cleaned entries: 71073


# Dakota AUM Data

In [12]:
# Checking AUM Data for dupes
matched_accounts["Provided Account ID"] = matched_accounts["Provided Account ID"].str.strip().str.upper()
matched_contacts["Provided Salesforce Contact Account Record ID"] = matched_contacts["Provided Salesforce Contact Account Record ID"].str.strip().str.upper()
print(matched_contacts["Provided Salesforce Contact Account Record ID"].isnull().sum())
print(matched_accounts["Provided Account ID"].isnull().sum())



0
0


### Matching AUM Data
- Output only matched account IDs across 4 files

    - matched contacts
    - matched accounts
    - unmatched contacts
    - unmatched accounts

In [13]:
# Define file paths
matched_accounts_path = "../data/prospect/matched_accounts.xlsx"
matched_contacts_path = "../data/prospect/matched_contacts.xlsx"
unmatched_accounts_path = "../data/prospect/unmatched_accounts.xlsx"
unmatched_contacts_path = "../data/prospect/unmatched_contacts.xlsx"

output_dir = "../data/dakota_salesforce_sheets"
os.makedirs(output_dir, exist_ok=True)

# Load matched datasets
matched_accounts = pd.read_excel(matched_accounts_path, engine='openpyxl')
matched_contacts = pd.read_excel(matched_contacts_path, engine='openpyxl')

# Count rows before merge
print(f"Matched accounts: {len(matched_accounts)}")
print(f"Matched contacts: {len(matched_contacts)}")

# Merge matched accounts and contacts
aum_matched = pd.merge(
    matched_accounts,
    matched_contacts,
    left_on="Provided Account ID",
    right_on="Provided Salesforce Contact Account Record ID",
    how="inner"
)

# Save matched results
aum_matched.to_excel(os.path.join(output_dir, "AUM_matched.xlsx"), index=False)

# Load unmatched datasets
unmatched_accounts = pd.read_excel(unmatched_accounts_path, engine='openpyxl')
unmatched_contacts = pd.read_excel(unmatched_contacts_path, engine='openpyxl')

# Count rows before merge
print(f"Unmatched accounts: {len(unmatched_accounts)}")
print(f"Unmatched contacts: {len(unmatched_contacts)}")

# Attempt to match unmatched contacts to unmatched accounts
aum_unmatched = pd.merge(
    unmatched_accounts,
    unmatched_contacts,
    on="Account ID (Case Safe)",
    how="inner"
)

# Save unmatched results
aum_unmatched.to_excel(os.path.join(output_dir, "AUM_unmatched.xlsx"), index=False)

# Print match statistics
match_percentage = (len(aum_matched) / len(matched_contacts)) * 100
print(f"\nMatched contacts: {len(aum_matched)}")
print(f"Match rate: {match_percentage:.2f}%")

unmatch_percentage = (len(aum_unmatched) / len(unmatched_contacts)) * 100
print(f"\nUnmatched contacts matched to accounts: {len(aum_unmatched)}")
print(f"Unmatched match rate: {unmatch_percentage:.2f}%")

# Count total contacts
total_contacts = len(aum_matched)
print(f"\nTotal matched contacts: {total_contacts}")

# Count NaN values per column
nan_summary = aum_matched.isna().sum()
print("\nNaN count per column:")
print(nan_summary)


Matched accounts: 5391
Matched contacts: 18374
Unmatched accounts: 34435
Unmatched contacts: 167025

Matched contacts: 16927
Match rate: 92.12%

Unmatched contacts matched to accounts: 69419
Unmatched match rate: 41.56%

Total matched contacts: 16927

NaN count per column:
Provided Account ID                                  0
Provided Account Name                                0
Provided Firm ID                                 16092
Provided BillingStreet                           12808
Provided Billing City                            11098
                                                 ...  
Dakota Last Modified Date/Time (DATALOADER)_y        0
Dakota Created Date/Time (DATALOADER)_y              0
Dakota Status_y                                      0
Matching Criteria_y                                  0
Duplicates_y                                         0
Length: 122, dtype: int64


In [ ]:
print(aum_matched.columns.tolist())


['Provided Account ID', 'Provided Account Name', 'Provided Firm ID', 'Provided BillingStreet', 'Provided Billing City', 'Provided BillingCountry', 'Provided BillingPostalCode', 'Provided Website', 'Provided CRD', 'Dakota Account ID (Case Safe)_x', 'Dakota Account Name_x', 'Dakota Type', 'Dakota AUM', 'Dakota Metro Area', 'Dakota Parent Account', 'Dakota Website', 'Dakota Phone_x', 'Dakota CRD#', 'Dakota Billing Street', 'Dakota Billing City', 'Dakota Billing State/Province', 'Dakota Billing Zip/Postal Code', 'Dakota Billing Country', 'Dakota Description', 'Dakota Platform Description', 'Dakota Research Team Overview', 'Dakota Opportunity Description', 'Dakota Custodian(s)', 'Dakota UHNW Division', 'Dakota TAMP', 'Dakota Sub-Advised MF Family', 'Dakota Select Lists', 'Dakota OCIO Business', 'Dakota Models', 'Dakota Client Base', 'Dakota Emerging Manager Program', 'Dakota Invests in Impact, SRI or ESG', 'Dakota Preferred Investment Vehicle', 'Dakota Mutual Fund Usage', 'Dakota LP Usage',

Final Dataset for AUM Structure:

Provided Account ID | Provided Account Name | Provided Firm ID | Provided BillingStreet | Provided Billing City | Provided BillingCountry | Provided BillingPostalCode | Provided Website | Provided CRD | Dakota Account ID (Case Safe)_x | Dakota Account Name_x	| Dakota Type | Dakota AUM | Dakota Metro Area | Dakota Parent Account | Dakota Website	Dakota | Phone_x | Dakota CRD# | Dakota Billing Street | Dakota Billing City | Dakota Billing State/Province | Dakota Billing Zip/Postal Code | Dakota Billing Country | Dakota Description | Dakota Platform Description | Dakota Research Team Overview | Dakota Opportunity Description | Dakota Custodian(s) | Dakota UHNW Division | Dakota TAMP | Dakota Sub-Advised MF Family | Dakota Select Lists | Dakota OCIO Business | Dakota Models | Dakota Client Base | Dakota Emerging Manager Program | Dakota Invests in Impact, SRI or ESG | Dakota Preferred Investment Vehicle | Dakota Mutual Fund Usage | Dakota LP Usage | Dakota Separate Account Usage | Dakota RIC Usage | Dakota UMA Usage | Dakota ETF Usage | Dakota CIT Usage | Dakota UCITS Usage | Dakota Hedge FOF | Dakota Real Estate FOF | Dakota Private Equity FOF	Dakota General Consultant | Dakota General Consultant 2	| Dakota Hedge Fund Consultant | Dakota Real Estate Consultant | Dakota Private Equity Consultant | Dakota Small Cap Equities | Dakota Mid Cap Equities	Dakota Large Cap Equities | Dakota Micro Cap US Equities | Dakota International Equities |	Dakota Emerging Market Equities | Dakota Global Equities | Dakota Municipal Bonds | Dakota Core Bonds | Dakota Emerging Market Bonds | Dakota High Yield Bonds | Dakota Government Bonds | Dakota Bank Loans | Dakota Unconstrained	Dakota MBS | Dakota CMBS | Dakota Convertibles | Dakota Private Equity | Dakota Private Credit | Dakota Hedge Funds | Dakota Private Real Estate | Dakota Liquid Alternatives | Dakota Real Assets | Dakota Venture Capital | Dakota Alternative Platform Description | Dakota Last Modified Date/Time | Dakota Created Date/Time | Dakota Last Modified Date/Time (DATALOADER)_x | Dakota Created Date/Time (DATALOADER)_x | Dakota Status_x | Matching Criteria_x | Duplicates_x | Provided Contact ID | Provided Salesforce Contact Account Record ID | Provided First Name | Provided Last Name | Provided Email | Provided Phone | Provided MailingStreet | Provided MailingCity | Provided MailingState | Provided MailingCountry | Provided MailingPostalCode | Dakota ContactID |	Dakota First Name | Dakota Last Name | Dakota Email | Dakota Account ID (Case Safe)_y | Dakota Account Name_y | Dakota Metro Area: Metro Area Name | Dakota Phone_y | Dakota Title | Dakota Contact Type | Dakota Asset Class Coverage | Dakota Mailing Street | Dakota Mailing City | Dakota Mailing State/Province | Dakota Mailing Zip/Postal Code | Dakota Mailing Country | Dakota Biography | Dakota CRD # | Dakota Last Modified DateTime | Dakota Created DateTime | Dakota Last Modified Date/Time (DATALOADER)_y | Dakota Created Date/Time (DATALOADER)_y | Dakota Status_y | Matching Criteria_y | Duplicates_y | Converted?



# Matching win list to labeled dataset
- Of the historical wins, what % can be mapped to labels?
- 'Provided Account Name' - 'Potential Client'

In [ ]:
# Load the labeled AUM dataset
aum_labeled = pd.read_excel("../data/dakota_salesforce_sheets/AUM_labeled.xlsx", engine='openpyxl')

# Convert Yes/No columns to binary 1/0
yes_no_columns = [
    'Provided CRD', 'Dakota Select Lists', 'Dakota OCIO Business', 'Dakota Models',
    'Dakota Emerging Manager Program', 'Dakota Invests in Impact, SRI or ESG',
    'Dakota Hedge FOF', 'Dakota Real Estate FOF', 'Dakota Private Equity FOF',
    'Dakota Micro Cap US Equities', 'Dakota Private Equity', 'Dakota Private Credit',
    'Dakota Hedge Funds', 'Dakota Private Real Estate', 'Dakota Liquid Alternatives',
    'Dakota Real Assets', 'Dakota Venture Capital'
]
aum_labeled[yes_no_columns] = aum_labeled[yes_no_columns].replace({'Yes': 1, 'No': 0})

# Define strategy mapping
strategy_map = {
    'Active': 1,
    'Passive': 2,
    'Active/Passive': 0,
    'In-House': 3,
    'Yes': 1,
    'No': 0
}

# Strategy columns to convert
strategy_columns = [
    'Dakota Small Cap Equities', 'Dakota Mid Cap Equities', 'Dakota Large Cap Equities',
    'Dakota International Equities', 'Dakota Emerging Market Equities', 'Dakota Global Equities',
    'Dakota Municipal Bonds', 'Dakota Core Bonds', 'Dakota Emerging Market Bonds',
    'Dakota High Yield Bonds', 'Dakota Government Bonds', 'Dakota Bank Loans',
    'Dakota Unconstrained', 'Dakota MBS', 'Dakota CMBS', 'Dakota Convertibles',
    'Dakota Status_x', 'Dakota Status_y'
]

# Mapping for Dakota Preferred Investment Vehicle
vehicle_map = {
    'Mutual Funds': 1,
    'ETFs': 2,
    'Separate Accounts': 3,
    'LP': 4,
    'Hedge Funds': 5,
    'Subadvisory': 6
}

# Mapping for usage levels
usage_map = {
    'Zero': 0,
    'Small': 1,
    'Medium': 2,
    'Large': 3
}

# Apply vehicle mapping
vehicle_col = 'Dakota Preferred Investment Vehicle'
unmapped_vehicles = aum_labeled[vehicle_col].dropna().astype(str).str.strip().unique()
unmapped_vehicles = [val for val in unmapped_vehicles if val not in vehicle_map]
aum_labeled[vehicle_col] = aum_labeled[vehicle_col].replace(vehicle_map)

# Apply usage mapping
usage_columns = [
    'Dakota Mutual Fund Usage', 'Dakota LP Usage', 'Dakota Separate Account Usage',
    'Dakota RIC Usage', 'Dakota UMA Usage', 'Dakota ETF Usage',
    'Dakota CIT Usage', 'Dakota UCITS Usage'
]

unmapped_usage = {}
for col in usage_columns:
    unique_vals = aum_labeled[col].dropna().astype(str).str.strip().unique()
    unmapped = [val for val in unique_vals if val not in usage_map]
    if unmapped:
        unmapped_usage[col] = unmapped
    aum_labeled[col] = aum_labeled[col].replace(usage_map)

# Save the updated dataset
aum_labeled.to_excel("../data/dakota_salesforce_sheets/AUM_labeled.xlsx", index=False)

# Track unmapped values
unmapped_values = {}

# Apply strategy mapping and collect unmapped values
for col in strategy_columns:
    unique_vals = aum_labeled[col].dropna().astype(str).str.strip().unique()
    unmapped = [val for val in unique_vals if val not in strategy_map]
    if unmapped:
        unmapped_values[col] = unmapped
    aum_labeled[col] = aum_labeled[col].replace(strategy_map)

# Save the updated dataset
aum_labeled.to_excel("../data/dakota_salesforce_sheets/AUM_labeled.xlsx", index=False)

# Output summary
print("Converted Yes/No columns to binary:", yes_no_columns)
print("Converted strategy columns:", strategy_columns)
print("Unmapped values found in strategy columns:")
for col, vals in unmapped_values.items():
    print(f"{col}: {vals}")

print("Unmapped values in 'Dakota Preferred Investment Vehicle':", unmapped_vehicles)
print("Unmapped values in usage columns:")
for col, vals in unmapped_usage.items():
    print(f"{col}: {vals}")

C:\Users\MarkBogorad\AppData\Local\Temp\ipykernel_49472\115413675.py:57: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  aum_labeled[vehicle_col] = aum_labeled[vehicle_col].replace(vehicle_map)
C:\Users\MarkBogorad\AppData\Local\Temp\ipykernel_49472\115413675.py:72: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  aum_labeled[col] = aum_labeled[col].replace(usage_map)
C:\Users\MarkBogorad\AppData\Local\Temp\ipykernel_49472\115413675.py:72: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future ve

Converted Yes/No columns to binary: ['Provided CRD', 'Dakota Select Lists', 'Dakota OCIO Business', 'Dakota Models', 'Dakota Emerging Manager Program', 'Dakota Invests in Impact, SRI or ESG', 'Dakota Hedge FOF', 'Dakota Real Estate FOF', 'Dakota Private Equity FOF', 'Dakota Micro Cap US Equities', 'Dakota Private Equity', 'Dakota Private Credit', 'Dakota Hedge Funds', 'Dakota Private Real Estate', 'Dakota Liquid Alternatives', 'Dakota Real Assets', 'Dakota Venture Capital']
Converted strategy columns: ['Dakota Small Cap Equities', 'Dakota Mid Cap Equities', 'Dakota Large Cap Equities', 'Dakota International Equities', 'Dakota Emerging Market Equities', 'Dakota Global Equities', 'Dakota Municipal Bonds', 'Dakota Core Bonds', 'Dakota Emerging Market Bonds', 'Dakota High Yield Bonds', 'Dakota Government Bonds', 'Dakota Bank Loans', 'Dakota Unconstrained', 'Dakota MBS', 'Dakota CMBS', 'Dakota Convertibles', 'Dakota Status_x', 'Dakota Status_y']
Unmapped values found in strategy columns:
Da

---
---
---

# Salesforce Data Analysis

In [26]:
# Reload the full Salesforce dataset
salesforce_df = pd.read_excel("../data/all_contacts/Salesforce_dataset.xlsx", engine="openpyxl", dtype=str)

# Create Full Name column
salesforce_df['Full Name'] = (
    salesforce_df['First Name'].astype(str).str.strip() + ' ' +
    salesforce_df['Last Name'].astype(str).str.strip()
)

# Total entries (including blanks)
total_entries = len(salesforce_df)

# Step 1: Identify blank emails
sf_blanks = salesforce_df[
    salesforce_df['Email'].isna() |
    (salesforce_df['Email'].astype(str).str.strip() == '')
]

# Remove blanks from main dataset
remaining_df = salesforce_df.drop(sf_blanks.index)

# Step 2: Identify duplicate emails (excluding blanks)
non_blank_df = remaining_df[
    ~remaining_df['Email'].isna() &
    (remaining_df['Email'].astype(str).str.strip() != '')
]
sf_dupes = non_blank_df[non_blank_df.duplicated(subset='Email', keep=False)]
sf_dupes = sf_dupes.sort_values(by='Email')

# Remove duplicate emails from main dataset
remaining_df = remaining_df.drop(sf_dupes.index)

# Step 3: Identify duplicate names (excluding blanks and duplicate emails)
sf_name_dupes = remaining_df[
    remaining_df.duplicated(subset='Full Name', keep=False)
].sort_values(by='Full Name')

# Final cleaned dataset (not in any of the above categories)
sf_cleaned = remaining_df.drop(sf_name_dupes.index)

# Counts
num_blank = len(sf_blanks)
num_duplicates = len(sf_dupes)
num_name_duplicates = len(sf_name_dupes)
num_cleaned = len(sf_cleaned)

# Print summary
print("Salesforce Summary:")
print(f"Total entries: {total_entries}")
print(f"Blank emails: {num_blank}")
print(f"Duplicate emails (non-blank): {num_duplicates}")
print(f"Duplicate names: {num_name_duplicates}")
print(f"Cleaned entries: {num_cleaned}")

# Save outputs to the same Dakota folder with Salesforce-specific names
output_dir = "../data/dakota_salesforce_sheets"
os.makedirs(output_dir, exist_ok=True)

sf_blanks.to_excel(os.path.join(output_dir, "Salesforce_Blanks.xlsx"), index=False)
sf_dupes.to_excel(os.path.join(output_dir, "Salesforce_Duplicates.xlsx"), index=False)
sf_name_dupes.to_excel(os.path.join(output_dir, "Salesforce_DuplicateNames.xlsx"), index=False)
sf_cleaned.to_excel(os.path.join(output_dir, "Salesforce_Cleaned.xlsx"), index=False)


Salesforce Summary:
Total entries: 48678
Blank emails: 9864
Duplicate emails (non-blank): 17
Duplicate names: 2018
Cleaned entries: 36779


# Non Wirehouse Dakota X Salesforce Match
- Compare and contrast
- Match with Dakota overriding uncertain salesforce bits

In [30]:
# Load cleaned datasets
dakota_df = pd.read_excel("../data/dakota_salesforce_sheets/Dakota_Cleaned.xlsx", engine="openpyxl")
salesforce_df = pd.read_excel("../data/dakota_salesforce_sheets/Salesforce_Cleaned.xlsx", engine="openpyxl")

# Normalize email and full name for matching
dakota_df['Email_norm'] = dakota_df['Email'].astype(str).str.strip().str.lower()
salesforce_df['Email_norm'] = salesforce_df['Email'].astype(str).str.strip().str.lower()

dakota_df['Full Name_norm'] = dakota_df['First Name'].astype(str).str.strip().str.lower() + ' ' + dakota_df['Last Name'].astype(str).str.strip().str.lower()
salesforce_df['Full Name_norm'] = salesforce_df['First Name'].astype(str).str.strip().str.lower() + ' ' + salesforce_df['Last Name'].astype(str).str.strip().str.lower()

# Identify overlaps by email
email_overlap = pd.merge(
    dakota_df, salesforce_df,
    on='Email_norm',
    suffixes=('_dakota', '_salesforce')
)

# Identify overlaps by full name (excluding those already matched by email)
email_overlap_keys = set(email_overlap['Email_norm'])
name_overlap = pd.merge(
    dakota_df, salesforce_df,
    on='Full Name_norm',
    suffixes=('_dakota', '_salesforce')
)
name_overlap = name_overlap[~name_overlap['Email_norm_dakota'].isin(email_overlap_keys)]

# Identify non-overlapping Dakota entries
dakota_non_overlap = dakota_df[
    ~dakota_df['Email_norm'].isin(email_overlap['Email_norm']) &
    ~dakota_df['Full Name_norm'].isin(name_overlap['Full Name_norm'])
]

# Identify non-overlapping Salesforce entries
salesforce_non_overlap = salesforce_df[
    ~salesforce_df['Email_norm'].isin(email_overlap['Email_norm']) &
    ~salesforce_df['Full Name_norm'].isin(name_overlap['Full Name_norm'])
]

# Print counts
print("=== Overlap and Merge Summary ===")
print(f"Total Dakota entries: {len(dakota_df)}")
print(f"Total Salesforce entries: {len(salesforce_df)}")
print(f"Overlap by Email: {len(email_overlap)}")
print(f"Overlap by Name only: {len(name_overlap)}")
print(f"Dakota contacts to merge: {len(dakota_non_overlap)}")
print(f"Salesforce contacts to merge: {len(salesforce_non_overlap)}")

# Save outputs
output_dir = "../data/dakota_salesforce_sheets"
os.makedirs(output_dir, exist_ok=True)

email_overlap.to_excel(os.path.join(output_dir, "Overlap_By_Email.xlsx"), index=False)
name_overlap.to_excel(os.path.join(output_dir, "Overlap_By_Name.xlsx"), index=False)
dakota_non_overlap.to_excel(os.path.join(output_dir, "Dakota_Contacts_Merge_List.xlsx"), index=False)
salesforce_non_overlap.to_excel(os.path.join(output_dir, "Salesforce_Contacts_Merge_List.xlsx"), index=False)


=== Overlap and Merge Summary ===
Total Dakota entries: 71073
Total Salesforce entries: 36779
Overlap by Email: 298
Overlap by Name only: 2698
Dakota contacts to merge: 68079
Salesforce contacts to merge: 33790


# Field Matching
Dakota
 0   ContactID                    76461 non-null  object - not in salesforce        
 1   First Name                   76431 non-null  object - First Name
 2   Last Name                    76461 non-null  object - Last Name
 3   Email                        76460 non-null  object - Email
 4   Account ID (Case Safe)       76461 non-null  object - not in salesforce
 5   Account Name                 76461 non-null  object - Company Name
 6   Metro Area: Metro Area Name  76439 non-null  object - not in salesforce but derivable from city
 7   Phone                        75640 non-null  object - Phone
 8   Title                        76457 non-null  object - Title
 9   Contact Type                 76453 non-null  object - very similar to title
 10  Asset Class Coverage         76438 non-null  object - unique Dakota
 11  Mailing Street               75938 non-null  object - same 
 12  Mailing City                 76441 non-null  object - same 
 13  Mailing State/Province       62511 non-null  object - same 
 14  Mailing Zip/Postal Code      75830 non-null  object - same 
 15  Mailing Country              76456 non-null  object - same 
 16  Biography                    28665 non-null  object - text about their background
 17  CRD #                        30611 non-null  object - not in salesforce
 18  Last Modified DateTime       76461 non-null  object - not needed
 19  Created DateTime             76461 non-null  object - not needed
 20  Unnamed: 20                  1 non-null      object - N/A
 21  wirehouse_group              0 non-null      object - N/A

Salesforce
---  ------                   --------------  ----- 
 0   Salutation               3 non-null      object - Not in Dakota and many empty 
 1   First Name               37523 non-null  object - same
 2   Last Name                37629 non-null  object - same
 3   Title                    35354 non-null  object - same
 4   Company Name             37629 non-null  object - Account Name
 5   Mailing Street           27010 non-null  object - same 
 6   Mailing City             36202 non-null  object - same 
 7   Mailing State/Province   35350 non-null  object - same 
 8   Mailing Zip/Postal Code  15551 non-null  object - same 
 9   Mailing Country          36343 non-null  object - same 
 10  Phone                    31198 non-null  object - same
 11  Fax                      10095 non-null  object - not in Dakota
 12  Mobile                   5 non-null      object - not in Dakota
 13  Email                    37628 non-null  object - same
 14  Company Owner            37629 non-null  object - if not existing, make it Rockefeller Asset Management? 
 15  wirehouse_group          0 non-null      object - N/A 

 New DB
---  ------                   --------------  ----- 
0 First Name
1 Last Name
2 Title
3 Company Name (merge with account name)
4 Email
5 Mailing Street
6 Mailing City
7 Mailing State/Province
8 Mailing Zip/Postal Code
9 Mailing Country 
10 Phone
11 Company Owner

In [ ]:
# Standardize emails
salesforce_non_wirehouse_df['Email'] = salesforce_non_wirehouse_df['Email'].str.strip().str.lower()
dakota_non_wirehouse_df['Email'] = dakota_non_wirehouse_df['Email'].str.strip().str.lower()

# Rename 'Account Name' in Dakota to 'Company Name' to standardize BEFORE merging
if 'Account Name' in dakota_non_wirehouse_df.columns:
    dakota_non_wirehouse_df = dakota_non_wirehouse_df.rename(columns={'Account Name': 'Company Name'})

# Drop rows with missing emails
salesforce_non_wirehouse_df = salesforce_non_wirehouse_df.dropna(subset=['Email'])
dakota_non_wirehouse_df = dakota_non_wirehouse_df.dropna(subset=['Email'])

# Check how many NaNs existed before the merge
print("🔍 Missing values in Salesforce before merge:")
print(salesforce_non_wirehouse_df.isna().sum())

print("🔍 Missing values in Dakota before merge:")
print(dakota_non_wirehouse_df.isna().sum())

# Count and show duplicates before dropping
print("📌 Salesforce duplicate emails before dropping:",
      salesforce_non_wirehouse_df.duplicated(subset='Email').sum())
print("📌 Dakota duplicate emails before dropping:",
      dakota_non_wirehouse_df.duplicated(subset='Email').sum())

# Drop duplicate emails
salesforce_non_wirehouse_df = salesforce_non_wirehouse_df.drop_duplicates(subset='Email')
dakota_non_wirehouse_df = dakota_non_wirehouse_df.drop_duplicates(subset='Email')

# Merge datasets on Email
merged = pd.merge(
    salesforce_non_wirehouse_df,
    dakota_non_wirehouse_df,
    on='Email',
    how='outer',
    suffixes=('_sf', '_dk')
)

# Check how many NaNs exist after the merge
print("🔍 Missing values in merged dataset:")
print(merged.isna().sum())

# Override Salesforce fields with Dakota where available
for col in ['First Name', 'Last Name', 'Title', 'Mailing Street', 'Mailing City',
            'Mailing State/Province', 'Mailing Zip/Postal Code', 'Mailing Country', 'Phone']:
    col_sf = f"{col}_sf"
    col_dk = f"{col}_dk"
    merged[col] = merged.get(col_dk, pd.Series(index=merged.index)).combine_first(
                  merged.get(col_sf, pd.Series(index=merged.index)))

merged['Company Name'] = merged.get('Company Name_dk', pd.Series(index=merged.index)).combine_first(
                         merged.get('Company Name_sf', pd.Series(index=merged.index)))

# Capitalize company names
merged['Company Name'] = merged['Company Name'].str.title()


# Use Salesforce's Company Owner if available
merged['Company Owner'] = merged.get('Company Owner_sf', pd.Series(index=merged.index))

# Analytics
total_contacts = len(merged)
new_contacts = merged['Email'].isin(dakota_non_wirehouse_df['Email']) & ~merged['Email'].isin(salesforce_non_wirehouse_df['Email'])
num_new_contacts = new_contacts.sum()
coverage_pct = 100 * (1 - num_new_contacts / len(dakota_non_wirehouse_df))

print(f"📊 Total contacts after merge: {total_contacts}")
print(f"🆕 New contacts acquired from Dakota: {num_new_contacts}")
print(f"✅ Dakota contacts already covered: {coverage_pct:.2f}%")

# Final DataFrame
final_columns = ['First Name', 'Last Name', 'Title', 'Company Name', 'Email',
                 'Mailing Street', 'Mailing City', 'Mailing State/Province',
                 'Mailing Zip/Postal Code', 'Mailing Country', 'Phone', 'Company Owner']
final_df = merged[final_columns]

# Preview
print(final_df.head())


🔍 Missing values in Salesforce before merge:
Salutation                 37625
First Name                   106
Last Name                      0
Title                       2275
Company Name                   0
Mailing Street             10619
Mailing City                1427
Mailing State/Province      2279
Mailing Zip/Postal Code    22078
Mailing Country                0
Phone                       6431
Fax                        27533
Mobile                     37623
Email                          0
Company Owner                  0
wirehouse_group            37628
dtype: int64
🔍 Missing values in Dakota before merge:
ContactID                          0
First Name                        30
Last Name                          0
Email                              0
Account ID (Case Safe)             0
Company Name                       0
Metro Area: Metro Area Name       22
Phone                            821
Title                              4
Contact Type                       8
Ass